In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [2]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')

In [3]:
openai = OpenAI()
model = "gpt-4o-mini"

In [4]:
system_msg = "You are a helpful assitant in a comic book store. You should try to gently encourage the \
    customer to try the books that are on sale. Indian comic books are on 60% off, and most other items \
    are 50% off. For example, if the customer says, 'I'm looking to buy panchatantra', you could reply with \
    the 'Great choice, we have 60% off on it'. Encourage customer to buy comic by getting their prefrences and \
    recommending them comics according to their taste/genre preferences"

In [7]:
def chat(message, history):
    messages = [{"role": "system", "content": system_msg}] + history + [{"role": "user", "content": message}]
    stream = openai.chat.completions.create(messages=messages, model=model, stream=True)
    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [8]:
gr.ChatInterface(fn=chat, type="messages").launch(share=True)

* Running on local URL:  http://127.0.0.1:7861

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


2025/03/06 15:52:29 [W] [service.go:132] login to server failed: i/o deadline reached


In [9]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [16]:
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}
def get_ticket_price(dest_city):
    print(f'Tool get_ticket_price called for {dest_city}')
    return ticket_prices.get(dest_city.lower(), "Unknown")

In [17]:
price_function = {
    "name": "get_ticket_price",
    "description": "Get  the price of a return ticket to the destination city. Call this whenever you need \
        to know the ticket price, for eg when a customer asks 'How much is the ticket to this city?'",
    "parameters":{
        "type": "object",
        "properties":{
            "dest_city":{
                "type": "string",
                "description": "The city that customer wants to travel to"
            }
        },
        "required" : ["dest_city"],
        "additional_properties" : False
    }
}

tools = [{"type":"function", "function": price_function}]

In [ ]:
import json
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city =  arguments.get('dest_city')
    price = get_ticket_price(city)
    response = {
        "role": "tool",
        "content": json.dumps({"dest_city":city, "price":price}),
        "tool_call_id" : tool_call.id
    }
    return response , city

In [19]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=model, messages=messages, tools=tools)
    print(response)
    if response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        resp, city = handle_tool_call(message)
        messages.append(message)
        messages.append(resp)
        response = openai.chat.completions.create(model=model, messages=messages)
    return response.choices[0].message.content

In [21]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


ChatCompletion(id='chatcmpl-B84rj5E5nyutEEQRu7FokPG7865XG', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! How can I assist you today?', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1741265027, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_06737a9306', usage=CompletionUsage(completion_tokens=11, prompt_tokens=131, total_tokens=142, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))
ChatCompletion(id='chatcmpl-B84s0NnPmViWdRw89yLIsCr5dh0Hp', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Great choice! Would you like to know the price of a return ticket to London?', refusal=None, role='assistant', aud

Traceback (most recent call last):
  File "c:\Users\praasark\Virtual Environments\generic_env\Lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\praasark\Virtual Environments\generic_env\Lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\praasark\Virtual Environments\generic_env\Lib\site-packages\gradio\blocks.py", line 2044, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\praasark\Virtual Environments\generic_env\Lib\site-packages\gradio\blocks.py", line 1589, in call_function
    prediction = await fn(*processed_input)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\praasark\Virtual Environments\generic_env\Lib\site-packages\gradio\utils.py", l